In [ ]:
# Initial imports - some redundancies may be present
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import History
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import math
from tensorflow import py_function, double
import requests
import gc
import json
import tensorflow_addons as tfa
import random

# Set up number of features 
numFeatures = 100*7*4

# Checkpoint
print("Checkpoint: initialization complete")

### Custom preprocessing layer

In [ ]:
class PreprocessingLayer(Layer):
    def __init__(self, **kwargs):
        super(PreprocessingLayer, self).__init__(**kwargs) 

    def build(self, input_shape):
        self.temporalPenalty = self.add_weight(name='temporalPenalty',
                                    initializer=tf.keras.initializers.RandomUniform(minval=50, maxval=100),
                                    shape=(1,),
                                    trainable=True)

        super(PreprocessingLayer, self).build(input_shape)

    #create the layer operation here
    def call(self, x):
        # Reshape the data
        x = keras.layers.Reshape((700, 4))(x)
        
        # Caclulate the distances and grab the temperatures
        distances = tf.math.sqrt(tf.math.add_n([tf.math.square(x[:,:,0]), tf.math.square(x[:,:,1]), tf.math.multiply(self.temporalPenalty, tf.math.square(x[:,:,2]))]))
        temperatures = x[:, :, 3]
    
        # Sort the data by distances
        sortOrder = tf.argsort(distances, direction='ASCENDING')
        sortOrder = keras.layers.Reshape((700,))(sortOrder)
        sortedDistances = tf.gather(distances, sortOrder, batch_dims=1, axis=-1)
        sortedTemperatures = tf.gather(temperatures, sortOrder, batch_dims=1, axis=-1)

        # Prepare the features
        features = tf.reshape(tf.stack([sortedTemperatures, sortedDistances], axis=2),  (-1, 1400, 1))
        features = features[:, 0:200, :]
        features = tf.convert_to_tensor(features)
        return features
    
#     def compute_output_shape(self, input_shape):
#         return (None, 100, 3)

## Model set up ## 

In [ ]:
def create_model():
    # Preprocessing step
    x_input = tf.keras.layers.Input(shape=(numFeatures, )) # or numFeatures // 2 for only temperatures
    features = keras.layers.Reshape((700, 4))(x_input)
    features = PreprocessingLayer()(features)
    
    # LSTM Step
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(features)
    x = tf.keras.layers.Dense(1, activation='linear')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "LSTMLite")

    # Model parameters
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='mae', optimizer=opt, metrics=['mae', 'mse'])
    return model

# Create the model
model = create_model()
model.summary()

### Train the model

In [ ]:
# Train the network
modelCheckpoint = keras.callbacks.ModelCheckpoint("Models/LSTMLite.h5", 
                                                  save_best_only=True, 
                                                  save_weights_only=True,
                                                  monitor='val_loss', 
                                                  mode='min')

# Set up parameters for storing history
h = tf.keras.callbacks.History()
loss = np.empty(0);
val_loss = np.empty(0);
trainable_weight = np.empty(0);

# Load weights here if this is a second learning run
#model.load_weights("Models/LSTMLite.h5")

counter = 0

# Training step
# Model was trained using manual epochs; it ran one epoch for each file loaded, then closed again
# Intention was to maximise shuffling and support generalization
for j in range(0, 100):
    indexArray = list(range(0, 15))
    # Shuffle open order for better generalization
    random.shuffle(indexArray)
    for i in indexArray:
        
        # If this isn't the first run, load the previous weights
        if counter > 0:
            model.load_weights("Models/LSTMLite2.h5")

        # Load the training set
        X_train = np.load("X_train_AllN_test" + str(i) + ".npy", mmap_mode='r')
        y_train = np.load("Y_train_AllN_test" + str(i) + ".npy", mmap_mode='r')

        # Train
        h = model.fit(X_train, y_train, batch_size=1024, validation_split=0.5, epochs=1, callbacks=[modelCheckpoint], shuffle=True)
        print(model.trainable_variables[0])
        
        # Update tracking
        loss = np.append(loss, h.history['loss'])
        val_loss = np.append(val_loss, h.history['val_loss'])
        trainable_weight = np.append(trainable_weight, model.trainable_variables[0])
        
        #Save the model
        model.save_weights("Models/LSTMLite2.h5")

        counter = counter + 1;

        del X_train, y_train
        gc.collect()

# summarize history for loss
plt.figure(0)
plt.plot(loss)
plt.plot(val_loss)
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

### Test the model ###

In [ ]:
# Evaluate on the test set

# Run the python garbage collector, which will delete old objects on the GPU.
# This is useful after modifying the model and restarting training.
gc.collect()

# Set up arrays and load weights
y_true = np.empty(0)
y_pred = np.empty(0)
model.load_weights("Models/LSTMLite2.h5")

# Loop through all testing files
for i in range(0, 9):
    print("Up to file " + str(i))
    
    # Load the first testing tiles
    X_test = np.load("X_test_AllN_" + str(i) + ".npy", mmap_mode='r')
    y_test = np.load("Y_test_AllN_" + str(i) + ".npy", mmap_mode='r')

    # Performance metrics
    performance_metrics = model.evaluate(X_test, y_test, verbose=1)
    print('metrics' + str(performance_metrics))

    # Append the y values to the growing arrays above
    y_pred = np.append(y_pred, model.predict(X_test))
    y_true = np.append(y_true, y_test)
    
    del X_test, y_test
    gc.collect()

# Get the actual MAE and RMSE for the whole set
# MAE
maeActual = np.mean(np.absolute(y_pred-y_true))
print("MAE in degC = " + str(maeActual))

# RMSE
rmse = np.sqrt(np.mean(np.square(y_pred-y_true)))
print("RMSE in degC = " + str(rmse))

## Plot the history ##

In [ ]:
# Plot the fit
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['train', 'test'], loc='upper left')
plt.show()